In [10]:
import os, sys, time
import numpy as np
import matplotlib.pyplot as plt
import scipy
import healpy as hp
import astropy.io.fits as fits
import camb
import jax
import jax.numpy as jnp
import jax.scipy as jsp
import jax_healpy as jhp
import numpyro
from functools import partial
import micmac as micmac
# from mcmc_tools import *

from jax import config
config.update("jax_enable_x64", True)


In [11]:
# file_ver = 'full_v8_Gchain_SO_noise_v5a' # Mini-test long run 5 iterations with chain-in-code -- 10 Fisher start from params + sampling for r + C_approx only lensing ; minimal number of E-modes put to 0 artificially ; r=0.001, start=10**(-8)

working_directory_path = '/Users/mag/Documents/PHD1Y/Space_Work/Pixel_non_P2D/MICMAC/test_playground/validation_chain_v2/'
directory_save_file = working_directory_path + 'save_directory/'
directory_toml_file = working_directory_path + 'toml_params/'

path_toml_file = directory_toml_file + 'test_full_chain_v1a.toml'

In [12]:
micmac.Sampling_functions(nside=64, lmax=128, nstokes=2, frequency_array=0, freq_inverse_noise=0)

In [13]:
MICMAC_sampler_obj = micmac.create_MICMAC_sampler_from_toml_file(path_toml_file)

In [14]:
jnp.log2(64)

Array(6., dtype=float64)

In [15]:
2**6

64

In [16]:
sys.path.append(os.path.dirname(os.path.abspath('')))
sys.path.append(os.path.dirname(os.path.dirname(os.path.abspath('')))+'/tutorials/')
from func_tools_for_tests import *
# from get_freq_maps_SO_64 import *
from get_freq_maps_SO_64_lower_noise import *

OMP: Info #276: omp_set_nested routine deprecated, please use omp_set_max_active_levels instead.


Shape for input frequency maps : (6, 2, 49152)
Shape for input cmb maps : (6, 2, 49152)


In [17]:
input_freq_maps, theoretical_red_cov_r0_total, theoretical_red_cov_r1_tensor = MICMAC_sampler_obj.generate_input_freq_maps_from_fgs(freq_maps_fgs, return_only_freq_maps=False)

theoretical_r0_total = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r0_total)#[partial_indices_polar,:]
theoretical_r1_tensor = micmac.get_c_ells_from_red_covariance_matrix(theoretical_red_cov_r1_tensor)#[partial_indices_polar,:]


CMB_c_ell = np.zeros((3,MICMAC_sampler_obj.lmax+1))
CMB_c_ell[:,MICMAC_sampler_obj.lmin:] = (theoretical_r0_total + MICMAC_sampler_obj.r_true*theoretical_r1_tensor)

red_cov_matrix = get_reduced_matrix_from_c_ell(CMB_c_ell)[MICMAC_sampler_obj.lmin:,...]

Calculating spectra from CAMB !
Calculating spectra from CAMB !


In [19]:
jitted_single_Metropolis_Hasting_step_r = jax.jit(micmac.single_Metropolis_Hasting_step, static_argnames=['log_proba'])

In [22]:
jitted_single_Metropolis_Hasting_step_r(random_PRNGKey=jax.random.PRNGKey(0)+3, old_sample=10**(-3), 
                                                    step_size=10**(-4), log_proba=MICMAC_sampler_obj.get_conditional_proba_C_from_r, 
                                                    red_sigma_ell=red_cov_matrix, theoretical_red_cov_r1_tensor=theoretical_red_cov_r1_tensor, theoretical_red_cov_r0_total=theoretical_red_cov_r0_total)

Array(0.00087017, dtype=float64)

In [31]:
def func_test(x,y): 
    print(y)
    return y

jlx.fori_loop(0,10,func_test,5)

Traced<ShapedArray(int64[], weak_type=True)>with<DynamicJaxprTrace(level=1/0)>


Array(5, dtype=int64, weak_type=True)

In [25]:
z = 5
for i in range(10):
    z += i

In [26]:
z

50

In [36]:
a_test_0 = np.arange(4).reshape((2,2))
b_test_0 = np.arange(4).reshape((2,2))



In [46]:
c_test = a_test_0.reshape(([1]+list(a_test_0.shape)))

In [44]:
jnp.array([[1]]).shape

(1, 1)

In [50]:
d_test = np.vstack([c_test,c_test])
np.vstack([d_test,c_test]).shape

(3, 2, 2)